In [8]:
import pandas as pd
import numpy as np
import random
import os
import utm
import matplotlib.pyplot as plt
from utils.preprocessing import preprocess
from utils.from_latlon import from_latlon
import math
from sklearn.linear_model import LinearRegression
import tkinter as tk
from tkinter import filedialog
from tkinter.simpledialog import askstring

In [9]:
routine = 'training'
dimensions = ['xOffset', 'yOffset']
sm = 150 # maximum signal difference between min and max, used for calibration

In [10]:
def preprocess_sim_data(sim_data, freq, tower_locs, routine):
    
    sim_dat_filt, predictors = preprocess(sim_data, freq, routine)
     
    # Calculate easting and northing from lat long
    sim_dat_filt['easting'], sim_dat_filt['northing'], sim_dat_filt['zone_num'], sim_dat_filt['zone_letter'] = from_latlon(sim_dat_filt['POINT_Y'].values, sim_dat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = tower_locs.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    sim_dat_filt['xOffset'] = sim_dat_filt['easting'] - sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['yOffset'] = sim_dat_filt['northing'] - sim_dat_filt['TowerID'].map(point_y).fillna(0)
    
    sim_dat_filt['easting_of_tower'] = sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['northing_of_tower'] = sim_dat_filt['TowerID'].map(point_y).fillna(0)

    return sim_dat_filt, predictors

In [11]:
# User input of data paths and temporal resolution

# Specify data paths
train_data = 'SBTF_data/input_data/train_test_data/train_data.xlsx'
test_data = 'SBTF_data/input_data/train_test_data/test_data.xlsx'
radio_tower_xy_path = 'SBTF_data/input_data/radio_tower_locations/RTEastNorth_1group.xlsx'
model_save_path = 'SBTF_data\ml4rt_output\predictions\sample_size_analysis_angulation_w_distance'

freq = '3min' # Frequency of data

In [12]:
def calculate_delta_g(s1, s2, sm):
    delta_g = (s1-s2)/sm
    return delta_g

def calculate_offset_angle(delta_g):
    bearing = math.acos(delta_g) * 90/math.pi
    return bearing

In [13]:
# Get training data
train_data = pd.read_excel(train_data)
train_data['DateAndTime'] = pd.to_datetime(train_data['DateAndTime'])

# Get testing data
test_data = pd.read_excel(test_data)
test_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get tower locations
tower_locs = pd.read_excel(radio_tower_xy_path)

In [14]:
# Create test dataset as per train_model_h2o method
test_data_preproc, predictors_test = preprocess_sim_data(test_data, freq, tower_locs, routine)
train_data_preproc, predictors_train = preprocess_sim_data(train_data, freq, tower_locs, routine)

c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x0000017CFF2D36A0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x0000017CFF2D36A0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])


In [25]:
unique_ids = train_data_preproc['Point_ID'].unique().tolist()

# For loop to sample training data, train, and test the models and export the result
train_data_length = len(unique_ids)
n = 5
run = 10

while n <= train_data_length:
    # Create a random subset of the training dataframe based on random Point_IDs, with the number of points equal to n
    random.shuffle(unique_ids)
    subset = unique_ids[:n]
    train_data_preproc_sample = train_data_preproc[train_data_preproc['Point_ID'].isin(subset)]

    test_data_preproc_n = test_data_preproc.copy()
    results = []

    # do the thing
    # Make distance values absolute
    train_data_preproc_sample['xOffset_abs'] = train_data_preproc_sample['xOffset'].abs()
    train_data_preproc_sample['yOffset_abs'] = train_data_preproc_sample['yOffset'].abs()

    # Perform linear regression for each antenna
    train_data_filtered = train_data_preproc_sample[train_data_preproc_sample['ant1_mean'] != 0]
    x = train_data_filtered[['ant1_mean']]
    y = train_data_filtered[['yOffset_abs']]
    model_ant1 = LinearRegression().fit(x, y)

    train_data_filtered = train_data_preproc_sample[train_data_preproc_sample['ant2_mean'] != 0]
    x = train_data_filtered[['ant2_mean']]
    y = train_data_filtered[['xOffset_abs']]
    model_ant2 = LinearRegression().fit(x, y)

    train_data_filtered = train_data_preproc_sample[train_data_preproc_sample['ant3_mean'] != 0]
    x = train_data_filtered[['ant3_mean']]
    y = train_data_filtered[['yOffset_abs']]
    model_ant3 = LinearRegression().fit(x, y)

    train_data_filtered = train_data_preproc_sample[train_data_preproc_sample['ant4_mean'] != 0]
    x = train_data_filtered[['ant4_mean']]
    y = train_data_filtered[['xOffset_abs']]
    model_ant4 = LinearRegression().fit(x, y)

    # Also create an average model
    # Extract coefficients from individual models
    coefficients_ant1 = np.array([model_ant1.coef_[0][0], model_ant1.intercept_[0]])
    coefficients_ant2 = np.array([model_ant2.coef_[0][0], model_ant2.intercept_[0]])
    coefficients_ant3 = np.array([model_ant3.coef_[0][0], model_ant3.intercept_[0]])
    coefficients_ant4 = np.array([model_ant4.coef_[0][0], model_ant4.intercept_[0]])

    # Calculate average coefficients
    average_coefficients = np.mean([coefficients_ant1, coefficients_ant2, coefficients_ant3, coefficients_ant4], axis=0)

    # Create a new Linear Regression model with average coefficients
    model_ant_average = LinearRegression()
    model_ant_average.coef_ = np.array([average_coefficients[0]])
    model_ant_average.intercept_ = np.array([average_coefficients[1]])

    ## Calculate total paired antenna values to find maximum sum, only if both values do not equal zero.

    # Add 'ant_1_2_sum' column
    test_data_preproc_n['ant_1_2_sum'] = test_data_preproc_n.apply(lambda row: row['ant1_mean'] + row['ant2_mean'] if row['ant1_mean'] != 0 and row['ant2_mean'] != 0 else 'na', axis=1)

    # Add 'ant_2_3_sum' column
    test_data_preproc_n['ant_2_3_sum'] = test_data_preproc_n.apply(lambda row: row['ant2_mean'] + row['ant3_mean'] if row['ant2_mean'] != 0 and row['ant3_mean'] != 0 else 'na', axis=1)

    # Add 'ant_3_4_sum' column
    test_data_preproc_n['ant_3_4_sum'] = test_data_preproc_n.apply(lambda row: row['ant3_mean'] + row['ant4_mean'] if row['ant3_mean'] != 0 and row['ant4_mean'] != 0 else 'na', axis=1)

    # Add 'ant_4_1_sum' column
    test_data_preproc_n['ant_4_1_sum'] = test_data_preproc_n.apply(lambda row: row['ant4_mean'] + row['ant1_mean'] if row['ant4_mean'] != 0 and row['ant1_mean'] != 0 else 'na', axis=1)

    #### Calculate the bearings from the tower

    # List of antenna names
    antenna_names = ['ant_1_2_sum', 'ant_2_3_sum', 'ant_3_4_sum', 'ant_4_1_sum']

    # Create a new column 'bearing' in the DataFrame
    test_data_preproc_n['bearing'] = None

    # Iterate through the DataFrame
    for index, row in test_data_preproc_n.iterrows():
        # Extract values from the new columns and handle 'na' values
        values = [float(row['ant_1_2_sum']) if row['ant_1_2_sum'] != 'na' else float('-inf'),
                float(row['ant_2_3_sum']) if row['ant_2_3_sum'] != 'na' else float('-inf'),
                float(row['ant_3_4_sum']) if row['ant_3_4_sum'] != 'na' else float('-inf'),
                float(row['ant_4_1_sum']) if row['ant_4_1_sum'] != 'na' else float('-inf')]
        
        # Check if all values are 'na'; if yes, skip the row
        if all(val == float('-inf') for val in values):
            continue
        
        # Find the column name with the highest value
        max_column = max(zip(values, antenna_names))[1]
            
        if max_column == "ant_1_2_sum":
            if row.ant1_mean >= row.ant2_mean:
                delta_g = calculate_delta_g(row.ant1_mean, row.ant2_mean, sm)
                offset_angle = calculate_offset_angle(delta_g)
                bearing = offset_angle
                
                # print(f'delta_g = {delta_g}')
                # print(f'offset_angle from ant 1 = {offset_angle}')
                # print(f'bearing ant 1 = {bearing}')

            else:
                delta_g = calculate_delta_g(row.ant2_mean, row.ant1_mean, sm)
                offset_angle = calculate_offset_angle(delta_g)          
                bearing = 90 - offset_angle

        elif max_column == "ant_2_3_sum":
            if row.ant2_mean >= row.ant3_mean:
                delta_g = calculate_delta_g(row.ant2_mean, row.ant3_mean, sm)
                offset_angle = calculate_offset_angle(delta_g)
                bearing = 90 + offset_angle

            else:
                delta_g = calculate_delta_g(row.ant3_mean, row.ant2_mean, sm)
                offset_angle = calculate_offset_angle(delta_g)          
                bearing = 180 - offset_angle

        elif max_column == "ant_3_4_sum":
            if row.ant3_mean >= row.ant4_mean:
                delta_g = calculate_delta_g(row.ant3_mean, row.ant4_mean, sm)
                offset_angle = calculate_offset_angle(delta_g)
                bearing = 180 + offset_angle

            else:
                delta_g = calculate_delta_g(row.ant4_mean, row.ant3_mean, sm)
                offset_angle = calculate_offset_angle(delta_g)          
                bearing = 270 - offset_angle

        elif max_column == "ant_4_1_sum":
            if row.ant4_mean >= row.ant1_mean:
                delta_g = calculate_delta_g(row.ant4_mean, row.ant1_mean, sm)
                offset_angle = calculate_offset_angle(delta_g)
                bearing = 270 + offset_angle

            else:
                delta_g = calculate_delta_g(row.ant1_mean, row.ant4_mean, sm)
                offset_angle = calculate_offset_angle(delta_g)          
                bearing = 360 - offset_angle
        
        
        # Correct bearings for magnetic declination and handle those that go over 360 degrees
        bearing = bearing + 7.6
        if bearing >= 360:
            bearing = bearing - 360
        else:
            pass

        # Assign the calculated bearing to the 'bearing' column
        test_data_preproc_n.at[index, 'bearing'] = bearing


    # Convert non-numeric values to NaN
    numeric_columns = ['ant_1_2_sum', 'ant_2_3_sum', 'ant_3_4_sum', 'ant_4_1_sum']
    test_data_preproc_n[numeric_columns] = test_data_preproc_n[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Calculate the maximum value across columns while handling NaN values
    test_data_preproc_n['max_value'] = np.nanmax(test_data_preproc_n[numeric_columns].values, axis=1) / 2

    # Calculate locations using the linear regression along bearings (without biangulation)

    # New line to create a complete copy of file for estimates with linear regression method
    mech_wLR_estimates = test_data_preproc_n.copy()

    # List of antenna names
    antennas = ['ant1_mean', 'ant2_mean', 'ant3_mean', 'ant4_mean']

    for index, row in mech_wLR_estimates.iterrows():
        # First calculate appropriate bearing and infer distance using linear model
        if row['bearing'] != None:
            distance = model_ant_average.predict(np.array([[row['max_value']]]))[0]
            bearing = row['bearing']
            mech_wLR_estimates.at[index, 'Mech_method'] = "Multiple antenna, single tower"

        elif row['bearing'] == None:
            # Extract values from the new columns and handle 'na' values
            values = [float(row['ant1_mean']) if row['ant1_mean'] != 'na' else float('-inf'),
                    float(row['ant2_mean']) if row['ant2_mean'] != 'na' else float('-inf'),
                    float(row['ant3_mean']) if row['ant3_mean'] != 'na' else float('-inf'),
                    float(row['ant4_mean']) if row['ant4_mean'] != 'na' else float('-inf')]

            mech_wLR_estimates.at[index, 'Mech_method'] = "Single antenna, single tower"

            # Find the column name with the highest value
            max_single_ant = max(zip(values, antennas))[1]

            if max_single_ant == "ant1_mean":
                bearing = 0
                distance = model_ant1.predict(np.array([[row['ant1_mean']]]))[0]

            elif max_single_ant == "ant2_mean":
                bearing = 90
                distance = model_ant2.predict(np.array([[row['ant2_mean']]]))[0]

            elif max_single_ant == "ant3_mean":
                bearing = 180
                distance = model_ant3.predict(np.array([[row['ant3_mean']]]))[0]

            elif max_single_ant == "ant4_mean":
                bearing = 270
                distance = model_ant4.predict(np.array([[row['ant4_mean']]]))[0]

        else:
            pass
        
        # Correct bearings for magnetic declination and handle those that go over 360 degrees
        bearing = bearing + 7.6
        if bearing >= 360:
            bearing = bearing - 360
        else:
            pass
        
        # Estimating new location using trigonometry
        bearing_rad = math.radians(bearing)

        # Calculate new coordinates
        new_easting = row['easting_of_tower'] + distance * math.sin(bearing_rad)
        new_northing = row['northing_of_tower'] + distance * math.cos(bearing_rad)

        # Assigning the new coordinates to the 'new_easting' and 'new_northing' columns in mech_wLR_estimates
        mech_wLR_estimates.at[index, 'easting_pred'] = new_easting
        mech_wLR_estimates.at[index, 'northing_pred'] = new_northing


    mech_wLR_estimates = mech_wLR_estimates.groupby(['DateTime', 'TagID']).agg({
        'easting': 'first',
        'northing': 'first',
        'easting_pred': 'mean', 
        'northing_pred': 'mean', 
        'Data_type': 'first'
    }).reset_index()

    mech_wLR_estimates['easting_error'] = mech_wLR_estimates['easting_pred'] - mech_wLR_estimates['easting']
    mech_wLR_estimates['northing_error'] = mech_wLR_estimates['northing_pred'] - mech_wLR_estimates['northing']

    mech_wLR_estimates['error_m'] = np.sqrt((mech_wLR_estimates['easting_error']) ** 2
                            + (mech_wLR_estimates['northing_error']) ** 2)

    # Save to excel
    filename = f"{n}_{run}_error_estimates_mech_w_linear_regression.xlsx"
    mech_wLR_estimates.to_excel(os.path.join(model_save_path, filename), index=False)

    print(f'Completed iteration: {n}')

    # Iterate
    n = n+5

C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_preproc_sample['xOffset_abs'] = train_data_preproc_sample['xOffset'].abs()
C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_preproc_sample['yOffset_abs'] = train_data_preproc_sample['yOffset'].abs()
C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:160: RuntimeWarning: All-NaN s

Completed iteration: 5


C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:160: RuntimeWarning: All-NaN slice encountered
  test_data_preproc_n['max_value'] = np.nanmax(test_data_preproc_n[numeric_columns].values, axis=1) / 2
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegressio

Completed iteration: 10


C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_preproc_sample['xOffset_abs'] = train_data_preproc_sample['xOffset'].abs()
C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_preproc_sample['yOffset_abs'] = train_data_preproc_sample['yOffset'].abs()
C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:160: RuntimeWarning: All-NaN s

...

Completed iteration: 175


C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_preproc_sample['xOffset_abs'] = train_data_preproc_sample['xOffset'].abs()
C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_preproc_sample['yOffset_abs'] = train_data_preproc_sample['yOffset'].abs()
C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:160: RuntimeWarning: All-NaN s

Completed iteration: 180


C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_19920\3636519922.py:160: RuntimeWarning: All-NaN slice encountered
  test_data_preproc_n['max_value'] = np.nanmax(test_data_preproc_n[numeric_columns].values, axis=1) / 2
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegressio

Completed iteration: 185
